![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [13]:
import pandas as pd
import numpy as np

from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

In [14]:
df_rental = pd.read_csv("rental_info.csv")

In [15]:
df_rental['rental_length'] = pd.to_datetime(df_rental['return_date']) - pd.to_datetime(df_rental['rental_date'])
df_rental['rental_length_days'] = df_rental['rental_length'].dt.days


In [16]:
df_rental["deleted_scenes"] =  np.where(df_rental["special_features"].str.contains("Deleted Scenes"), 1, 0)
df_rental["behind_the_scenes"] =  np.where(df_rental["special_features"].str.contains("Behind the Scenes"), 1, 0)


In [17]:
col_drop = ["special_features", "rental_length", "rental_length_days", "rental_date", "return_date"]

In [18]:
X = df_rental.drop(col_drop, axis=1)
y = df_rental['rental_length_days']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

In [19]:
lasso = Lasso(alpha=0.3, random_state=9)

lasso.fit(X_train, y_train)
lasso_coef = lasso.coef_

X_lasso_train, X_lasso_test = X_train.iloc[:, lasso_coef > 0], X_test.iloc[:, lasso_coef > 0]

In [20]:
model = LinearRegression()
model = model.fit(X_lasso_train, y_train)
y_test_pred = model.predict(X_lasso_test)
mse_lin_reg_lesso = mean_squared_error(y_test, y_test_pred)

param_dist = {'n_estimators': np.arange(1,101,1), 'max_depth':np.arange(1,11,1)}

rf = RandomForestRegressor()

rand_search = RandomizedSearchCV(rf, param_distributions=param_dist, cv=5, random_state=9)
rand_search.fit(X_train, y_train)
hyper_params = rand_search.best_params_

rf = RandomForestRegressor(n_estimators=hyper_params["n_estimators"], 
                           max_depth=hyper_params["max_depth"], 
                           random_state=9)
rf.fit(X_train,y_train)
rf_pred = rf.predict(X_test)
mse_random_forest= mean_squared_error(y_test, rf_pred)

In [21]:
best_model = rf

best_mse = mse_random_forest
print(best_mse)

2.225667528098759
